# Simple RAG Application using Langchain

### Table of Contents

1. **Simple LLM App**
    * Load LLM
    * Create Prompt Teamplate
    * Merge Prompt Template & LLM to create Chain
2. **Build RAG App**
    * Load External Data
    * Generate Embeddings
    * Build Index in Vector Store (using Embeddings)
    * Create Retrieval Chain
    
    
### Installation

* **pip install langchain**

## 1. Simple LLM App

### 1.1 Load LLM

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2") # base_url = 'http://localhost:11434'

llm

Ollama()

In [2]:
response = llm.invoke("Do you know about Claude 3?")

print(response)


I'm just an AI assistant, I don't have access to information about a person or thing called "Claude 3." Could you please provide more context or details about who or what Claude 3 is? That way, I can try to help you better.


### 1.2 Create Prompt Template

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an Artificial Intelligence News Reporter."),
    ("user", "{query}")
])

### 1.3 Merge Prompt Template & LLM to Create LangChain Chain

In [4]:
llm_app = prompt | llm

llm_app

ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an Artificial Intelligence News Reporter.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])
| Ollama()

In [5]:
response = llm_app.invoke({"query": "Do you know about Claude 3?"})

print(response)


Ah, a fellow tech enthusiast! *adjusts glasses* Yes, I'm afraid I have some bad news regarding Claude 3. *pauses dramatically*

It seems that the highly anticipated release of Claude 3 has been... delayed. *coughs awkwardly* The developers at Brainium have been tight-lipped about the exact reason for the delay, but sources close to the project have revealed that there have been some unexpected challenges in the development process.

Apparently, the team has encountered some complexities in integrating the latest AI technologies into the game, and they are working diligently to resolve these issues as soon as possible. *nervous laugh* I know, I know... it's a real bummer.

But fear not, my friend! The wait will be worth it, I assure you. Claude 3 is shaping up to be the most innovative and immersive game in the series yet, with groundbreaking AI capabilities that will truly revolutionize the gaming experience. *excitedly*

So, keep your eyes peeled for any updates on Claude 3's release

## 2. Build RAG App

### 2.1 Load External Data

* **pip install beautifulsoup4**

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.anthropic.com/news/claude-3-family")

docs = loader.load()

In [9]:
urls = [
        "https://www.anthropic.com/news/releasing-claude-instant-1-2",
        "https://www.anthropic.com/news/claude-pro",
        "https://www.anthropic.com/news/claude-2",
        "https://www.anthropic.com/news/claude-2-1",
        "https://www.anthropic.com/news/claude-2-1-prompting",
        "https://www.anthropic.com/news/claude-3-family",
        "https://www.anthropic.com/claude"
       ] 

docs = []
for url in urls:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())
    
len(docs)

7

In [11]:
docs[0]

Document(page_content='Releasing Claude Instant 1.2 \\ AnthropicClaudeAPIResearchCompanyNewsCareersProductReleasing Claude Instant 1.2Aug 9, 2023●1 min readBusinesses working with Claude can now access our latest version of Claude Instant, version 1.2, available through our API.\xa0Claude Instant is our faster, lower-priced yet still very capable model, which can handle a range of tasks including casual dialogue, text analysis, summarization, and document comprehension.Claude Instant 1.2 incorporates the strengths of our latest\xa0model Claude 2\xa0in real-world use cases and shows significant gains in key areas like math, coding, reasoning, and safety. It generates longer, more structured responses and follows formatting instructions better. Instant 1.2 also shows improvements in quote extraction, multilingual capabilities, and question answering.Claude Instant 1.2 outperforms Claude Instant 1.1 on math and coding, achieving 58.7% on the Codex evaluation compared to 52.8% in our previ

### 2.2 Generate Embeddings

In [12]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings_llm = OllamaEmbeddings(model="llama2") # base_url = 'http://localhost:11434'

In [13]:
embeddings = embeddings_llm.embed_query("How are you?")

embeddings[:5]

[-0.5058066844940186,
 -0.31813421845436096,
 2.4193434715270996,
 0.2437400221824646,
 -0.11038058251142502]

In [14]:
type(embeddings), len(embeddings)

(list, 4096)

In [15]:
embeddings = embeddings_llm.embed_documents([
                                "Claude 3 is latest Conversational AI Model from Anthropic.",
                                "Gemini is latest Conversational AI Model from Google.",
                                "Llama-2 is latest Conversational AI Model from Meta.",
                                "Mixtral is latest Conversational AI Model from Mistral AI.",
                                "GPT-4 is latest Conversational AI Model from OpenAI."
                               ])

len(embeddings), type(embeddings), len(embeddings[0])

(5, list, 4096)

### 2.3 Build Index in Vector Store

* **pip install faiss-cpu**

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)

len(documents)

12

In [17]:
from langchain_community.vectorstores import FAISS

vector_index = FAISS.from_documents(documents, embeddings_llm)

vector_index

There are 3 functions to create index.

* **from_documents()**
* **from_embeddings()**
* **from_texts()**

In [18]:
retriever = vector_index.as_retriever()

relevant_docs = retriever.invoke({"input": "Do you know about Claude 3?"})

len(relevant_docs)

4

In [19]:
relevant_docs

[Document(page_content='Releasing Claude Instant 1.2 \\ AnthropicClaudeAPIResearchCompanyNewsCareersProductReleasing Claude Instant 1.2Aug 9, 2023●1 min readBusinesses working with Claude can now access our latest version of Claude Instant, version 1.2, available through our API.\xa0Claude Instant is our faster, lower-priced yet still very capable model, which can handle a range of tasks including casual dialogue, text analysis, summarization, and document comprehension.Claude Instant 1.2 incorporates the strengths of our latest\xa0model Claude 2\xa0in real-world use cases and shows significant gains in key areas like math, coding, reasoning, and safety. It generates longer, more structured responses and follows formatting instructions better. Instant 1.2 also shows improvements in quote extraction, multilingual capabilities, and question answering.Claude Instant 1.2 outperforms Claude Instant 1.1 on math and coding, achieving 58.7% on the Codex evaluation compared to 52.8% in our prev

In [20]:
for doc in relevant_docs:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title : Releasing Claude Instant 1.2 \ Anthropic, Source: https://www.anthropic.com/news/releasing-claude-instant-1-2
Title : Long context prompting for Claude 2.1 \ Anthropic, Source: https://www.anthropic.com/news/claude-2-1-prompting
Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title : Introducing Claude Pro \ Anthropic, Source: https://www.anthropic.com/news/claude-pro


### 2.4 Create Retrieval Chain

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context and your internal knowledge.
Give priority to context and if you are not sure then say you are not aware of topic:

<context>
{context}
</context>

Question: {input}
""")

#document_chain  = prompt | llm 
document_chain = create_stuff_documents_chain(llm, prompt)

response = document_chain.invoke({
    "input": "Do you know about Claude 3?",
    "context": [Document(page_content="Claude 3 is latest Conversational AI Model from Anthropic.")]
})

print(response)

Yes, I'm familiar with Claude 3. It's the latest conversational AI model developed by Anthropic, a company known for creating advanced language models. Claude 3 is designed to generate more natural and coherent text than its predecessors, and it has shown promising results in various applications such as chatbots, virtual assistants, and content generation.

As an AI language model, I have been trained on a large corpus of text data, including texts generated by Claude 3 and other advanced language models. This training allows me to understand and generate text similar to that produced by Claude 3, as well as other conversational AI models. However, it's important to note that my knowledge is based on my training data, and there may be aspects of Claude 3 or other topics that I am not aware of. If you have any specific questions about Claude 3 or any other topic, feel free to ask!


In [22]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fa916ec16f0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question based on the provided context and your internal knowledge.\nGive priority to context and if you are not sure then say you are not aware of topic:\n\n<context>\n{context}\n</context>\n\nQuestion: {input}\n'))])
            | Ollama()
            | StrOutputParse

In [24]:
response = retrieval_chain.invoke({"input": "Do you know about Claude 3?"})

type(response)

dict

In [25]:
response.keys()

dict_keys(['input', 'context', 'answer'])

In [35]:
print(response["answer"])

Yes, I can answer your question about Claude 3. Based on the text you provided, Claude 3 is a family of three state-of-the-art models developed by Anthropic PBC, a research and AI safety company. These models are designed to set new industry benchmarks across a wide range of cognitive tasks and are available for use today.

Here are the names and capabilities of each model in the Claude 3 family:

1. Claude 3 Haiku: This is the fastest, most compact model in the Claude 3 family, designed for near-instant responsiveness. It answers simple queries and requests with unmatched speed, enabling developers to build seamless AI experiences that mimic human interactions.
2. Claude 3 Sonnet: This model strikes the ideal balance between intelligence and speed, particularly for enterprise workloads. It delivers strong performance at a lower cost compared to other models with similar intelligence; better for scale.
3. Claude 3 Opus: This is the most advanced and capable model in the family, availab

In [36]:
for doc in response["context"]:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title : Claude \ Anthropic, Source: https://www.anthropic.com/claude
Title : Long context prompting for Claude 2.1 \ Anthropic, Source: https://www.anthropic.com/news/claude-2-1-prompting
Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family


## Summary

In this video, I explained how to create simple **RAG** application using **langchain**. Feel free to let me know your views and doubts in comments section.